In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn

import h5py
print(torch.cuda.is_available())

True


In [3]:
import hls4ml
test = hls4ml.converters.get_supported_pytorch_layers()
print('MaxPool2d' in test)
print('MaxPool1d' in test)

print(test)


2023-12-15 21:24:10.567533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 21:24:11.321746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


True
True
['cat', 'concat', 'concatenate', 'add', 'mul', 'multiply', 'sub', 'subtract', 'fmin', 'minimum', 'fmax', 'maximum', 'Linear', 'Softmax', 'ReLU', 'LeakyReLU', 'Threshold', 'ELU', 'PReLU', 'Sigmoid', 'Tanh', 'BatchNorm2d', 'BatchNorm1d', 'Batch_norm', 'Flatten', 'View', 'MaxPool1d', 'MaxPool2d', 'AvgPool1d', 'AvgPool2d', 'Conv1d', 'Conv2d']


In [4]:
class LeNet5(nn.Module):
     def __init__(self,num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 4, stride = 4))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(144, 48)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(48, 24)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(24, num_classes)
        self.flatten=nn.Flatten()

     def forward(self, x):
        
        out = self.layer1(x)
        
        out = self.layer2(out)
       
        out=self.flatten(out)
        
        out = self.fc(out)
        
        out = self.relu(out)
        out = self.fc1(out)
        
        out = self.relu1(out)
        embedding = self.fc2(out)
       
        
        
        return embedding

In [5]:
model_pytorch=LeNet5(5)
model_pytorch.load_state_dict(torch.load('PytorchLeNet_4848_new.pt'))
model_pytorch.eval() 

LeNet5(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=144, out_features=48, bias=True)
  (relu): ReLU()
  (fc1): Linear(in_features=48, out_features=24, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=24, out_features=5, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

In [6]:
path = '/home/ferroelectric/sean/RHEED_4848_test6.h5'

with h5py.File(path, 'r') as h5:
    img = h5['growth_1']['spot_2'][1]
    img= img/img.max()
img_tensor = torch.tensor(img).unsqueeze(0).unsqueeze(0).float()
print(img_tensor.shape)
with torch.no_grad():
    output_pytorch = model_pytorch(img_tensor)
print(output_pytorch)

"""This is the scaled output, the proper math needs to be done to unscale it """


torch.Size([1, 1, 48, 48])
tensor([[  0.3999,  -7.2148,  -1.5861,  -2.9926, -10.6000]])


'This is the scaled output, the proper math needs to be done to unscale it '

In [7]:
#changing around reuse values

def print_dict(d, indent=0):
    align = 20
    for key, value in d.items():
        print('  ' * indent + str(key), end='')
        if isinstance(value, dict):
            print()
            print_dict(value, indent+1)
        else:
            print(':' + ' ' * (20 - len(key) - 2 * indent) + str(value))  


config = hls4ml.utils.config_from_pytorch_model(model_pytorch, inputs_channel_last=True,transpose_outputs=False,default_reuse_factor=2, granularity = "name")

config["LayerName"] = {}
config["LayerName"]["fc"] = {}
config["LayerName"]["fc1"] = {}
config["LayerName"]["fc2"] = {}

config["LayerName"]["fc"]["Strategy"] = "Resource"
config["LayerName"]["fc"]["ReuseFactor"] = 128
config["LayerName"]["fc1"]["Strategy"] = "Resource"
config["LayerName"]["fc1"]["ReuseFactor"] = 128
config["LayerName"]["fc2"]["Strategy"] = "Resource"
config["LayerName"]["fc2"]["ReuseFactor"] = 64

    
print_dict(config)

model_hls = hls4ml.converters.convert_from_pytorch_model(model_pytorch, [[None,1,48,48]], hls_config=config, output_dir='hls_rheed_4848_new', project_name='rheed', part='xcku035-fbva676-2-e', io_type='io_stream' )
model_hls.compile()



Model
  Precision:         ap_fixed<16,6>
  ReuseFactor:       2
  InputsChannelLast: True
  TransposeOutputs:  False
  Strategy:          Latency
LayerName
  fc
    Strategy:        Resource
    ReuseFactor:     128
  fc1
    Strategy:        Resource
    ReuseFactor:     128
  fc2
    Strategy:        Resource
    ReuseFactor:     64
Interpreting Model ...
Topology:
Layer name: layer1_0, layer type: Conv2D, input shape: [[None, 1, 48, 48]]
Layer name: layer1_1, layer type: BatchNormalization, input shape: [[None, 6, 44, 44]]
Layer name: layer1_2, layer type: Activation, input shape: [[None, 6, 44, 44]]
Layer name: layer1_3, layer type: MaxPooling2D, input shape: [[None, 6, 44, 44]]
Layer name: layer2_0, layer type: Conv2D, input shape: [[None, 6, 11, 11]]
Layer name: layer2_1, layer type: BatchNormalization, input shape: [[None, 16, 7, 7]]
Layer name: layer2_2, layer type: Activation, input shape: [[None, 16, 7, 7]]
Layer name: layer2_3, layer type: MaxPooling2D, input shape: [[None,